In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa 
import IPython.display as ipd
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn 
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import soundfile as sf
import random
from sklearn.preprocessing import Normalizer

In [2]:
paths = []
for dirname, _, filenames in os.walk("C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder (3)"):
    for filename in filenames:
        audio_path = os.path.join(dirname, filename)
        paths.append(audio_path)

In [3]:
file_paths = []
for path in paths:
    new_path = path.replace("\\","/")
    file_paths.append([new_path, new_path.split("/")[6]])

In [4]:
df = pd.DataFrame(file_paths)
df.columns = ["file name", "label"]

In [5]:
def feature_extractor(file):
    audio, sr = librosa.load(file)
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=100)
    scaled = np.mean(mfccs_features.T, axis = 0)
    return scaled

In [6]:
df

,file name,label
0,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,7masa
1,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,7masa
2,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,7masa
3,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,7masa
4,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,7masa
...,...,...
763,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,Zain
764,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,Zain
765,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,Zain
766,C:/Users/YAMAMA/OneDrive/سطح المكتب/New folder...,Zain


In [10]:
data = pd.DataFrame(columns=[list(range(100))])
for index, file in tqdm(df.iterrows()):
    scaled = feature_extractor(file[0])
    # data["label"] = file["label"]
    data.loc[index] = scaled

0it [00:00, ?it/s]C:\Users\YAMAMA\AppData\Local\Temp\ipykernel_13204\3039164541.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  scaled = feature_extractor(file[0])
1it [00:01,  1.24s/it]C:\Users\YAMAMA\AppData\Local\Temp\ipykernel_13204\3039164541.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  scaled = feature_extractor(file[0])
C:\Users\YAMAMA\AppData\Local\Temp\ipykernel_13204\3039164541.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use 

In [11]:
data["label"] = df["label"]

In [12]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,label
0,-279.504120,105.176651,22.293432,50.609032,-0.655371,37.712746,-46.792007,28.072260,-16.843035,13.662006,...,-0.519518,-0.472664,-0.715976,0.061389,-0.243055,-0.312579,-0.142246,-0.160081,-0.206230,7masa
1,-153.324768,126.449883,-5.462253,31.283978,-17.989059,29.179432,-43.341270,19.993721,-23.948509,12.839014,...,-0.397861,-0.812689,-0.250969,-0.496344,-0.083922,-0.116309,-0.687420,-0.823623,-0.772648,7masa
2,-291.813141,144.694733,12.609060,48.460121,-13.533754,21.293814,-28.518505,27.200939,-23.412363,12.875666,...,0.586474,-1.668057,-0.553369,0.063673,-1.485642,-1.187451,0.606059,0.157940,-1.623875,7masa
3,-246.131195,145.276093,-5.050489,31.080191,-18.580650,36.779171,-42.277649,23.503204,-19.990332,25.502209,...,-1.654703,-1.196852,-0.768617,-0.677539,-0.235271,0.174115,-0.474289,-1.567551,0.059383,7masa
4,-296.192169,152.808990,31.136812,46.293468,-13.814946,22.733232,-24.074972,17.702084,-8.482790,8.144162,...,-0.110874,0.024818,-0.157667,0.031578,0.143847,0.070500,-0.053603,-0.448507,-0.709511,7masa


In [4]:
data = pd.read_csv("C:\\Users\\YAMAMA\\OneDrive\\سطح المكتب\\classifier\\soundClassefierData.csv")

In [5]:
labelsss = data["label"].squeeze().unique()

In [6]:
labelsss = ['7masa', 'Abdelrahman', 'Geda', 'Haridy', 'Hossam', 'Khaled','Zain']

In [7]:
labelsss

['7masa', 'Abdelrahman', 'Geda', 'Haridy', 'Hossam', 'Khaled', 'Zain']

In [8]:
data["label"].iloc[78]

'Abdelrahman'

In [9]:
le = LabelEncoder()

In [10]:
X = np.array(data.drop(columns="label"))
y = np.array(data["label"]).reshape(-1)
y = np.array(pd.get_dummies(y))

In [11]:
pd.DataFrame(y)

,0,1,2,3,4,5,6
0,True,False,False,False,False,False,False
1,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False
3,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...
763,False,False,False,False,False,False,True
764,False,False,False,False,False,False,True
765,False,False,False,False,False,False,True
766,False,False,False,False,False,False,True


In [12]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [13]:
norm = Normalizer()
Xtrain = norm.fit_transform(Xtrain)

In [14]:
Xtrain

array([[ 1.96312133e-01, -8.24592623e-01,  4.54827171e-01, ...,
        -2.72327757e-04, -1.79755279e-03, -3.09580078e-03],
       [ 9.71386052e-01, -5.36286207e-02,  2.14975671e-01, ...,
         1.43059309e-03,  1.47190639e-03,  6.01328173e-04],
       [ 6.40458288e-01, -6.98411434e-01,  2.97809717e-01, ...,
         1.24453670e-03, -1.19775781e-03, -5.42039045e-04],
       ...,
       [ 6.73261336e-01, -5.83502558e-01,  4.06192229e-01, ...,
        -6.14118818e-04, -2.16070843e-03, -7.97738031e-04],
       [ 7.97863441e-01, -4.86155986e-01,  3.27513148e-01, ...,
        -1.38855102e-03, -1.13354985e-03, -4.04222307e-04],
       [ 3.39684199e-01, -6.48656150e-01,  6.39875047e-01, ...,
         2.81994699e-03,  1.43115953e-03, -2.49639846e-03]])

In [15]:
input_size = Xtrain.shape[1]

In [16]:
input_size

101

In [107]:
class modell(nn.Module):
    def __init__(self):
        super(modell, self).__init__()
        self.fc1 = nn.Linear(input_size,1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 7)
    def forward(self, X):
        X = torch.relu(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = torch.relu(self.fc3(X))
        X = torch.relu(self.fc4(X))
        
        return F.softmax(self.fc5(X), dim = 1)
        

In [108]:
Xtrain = torch.tensor(Xtrain, dtype = torch.float32)
ytrain = torch.tensor(ytrain, dtype = torch.float32)
Xtest = torch.tensor(Xtest, dtype = torch.float32)
ytest = torch.tensor(ytest, dtype = torch.float32)

C:\Users\YAMAMA\AppData\Local\Temp\ipykernel_18928\351060697.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xtrain = torch.tensor(Xtrain, dtype = torch.float32)
C:\Users\YAMAMA\AppData\Local\Temp\ipykernel_18928\351060697.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ytrain = torch.tensor(ytrain, dtype = torch.float32)
C:\Users\YAMAMA\AppData\Local\Temp\ipykernel_18928\351060697.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xtest = torch.tensor(Xtest, dtype = torch.float32)
C:\Users\YAMAMA\AppData\Local\Temp\ipykernel_189

In [109]:
train_data = TensorDataset(Xtrain, ytrain)
test_data = TensorDataset(Xtest, ytest)

In [110]:
train_data_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=16, shuffle=True)

In [111]:
# for row, label in tqdm(train_data_loader):
#     predict = model(row)
#     preds = torch.argmax(predict, dim =1)
#     print(torch.argmax(label, dim =1), preds)
#     break

In [112]:
import torch.optim as optim
model = modell()
optimizer  = optim.Adam(model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()

In [113]:
acc = []
epochs = 200
for epoch in range(epochs):
    for row, label in train_data_loader:
        optimizer.zero_grad()

        predict = model(row)
        loss = loss_fn(predict, torch.argmax(label, dim =1))
        loss.backward()
        optimizer.step()

        preds = torch.argmax(predict, dim =1)
        labels = torch.argmax(label, dim =1)
        accuracy = (preds == labels).sum().item() / len(labels)
        acc.append(accuracy)
    print(f"epoch loss: {loss}")

epoch loss: 1.7156368494033813
epoch loss: 1.6568609476089478
epoch loss: 1.499735951423645
epoch loss: 1.9014865159988403
epoch loss: 1.82882821559906
epoch loss: 1.221832036972046
epoch loss: 1.332207202911377
epoch loss: 1.328776240348816
epoch loss: 1.6522351503372192
epoch loss: 1.18718683719635
epoch loss: 1.1865434646606445
epoch loss: 1.1910861730575562
epoch loss: 1.4986923933029175
epoch loss: 1.513497233390808
epoch loss: 1.4621680974960327
epoch loss: 1.6434003114700317
epoch loss: 1.3442379236221313
epoch loss: 1.8311924934387207
epoch loss: 1.5947633981704712
epoch loss: 1.1654359102249146
epoch loss: 1.3320823907852173
epoch loss: 1.3321627378463745
epoch loss: 1.6580448150634766
epoch loss: 1.4964324235916138
epoch loss: 1.9814869165420532
epoch loss: 1.6653841733932495
epoch loss: 1.5004777908325195
epoch loss: 1.4978097677230835
epoch loss: 1.498589038848877
epoch loss: 1.8314071893692017
epoch loss: 1.165833592414856
epoch loss: 1.3320876359939575
epoch loss: 1.66298

In [114]:
accuracy = sum(acc)/len(acc)
accuracy

0.5451682692307692

In [65]:
Eval_acc_per_batch = []
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    Eval_acc_per_batch= []
    for inputs, labels in test_data_loader:
        outputs = model(inputs)
        prediction = torch.argmax(outputs,dim=1)
        acc = (prediction == torch.argmax(labels,dim=1)).sum().item() / len(labels)
        Eval_acc_per_batch.append(acc)
print(f"validation accuracy: {sum(Eval_acc_per_batch)/len(Eval_acc_per_batch):.4f}")

validation accuracy: 0.6950


In [87]:
file = feature_extractor("C:\\Users\\YAMAMA\\OneDrive\\سطح المكتب\\test\\7masa.wav")
file = torch.tensor([file], dtype = torch.float32)

In [88]:
file

tensor([[-3.9567e+02,  1.3488e+02,  1.2502e+01,  4.4080e+01,  4.9983e+00,
          2.0711e+01, -5.0114e+01,  2.7795e+01, -1.6360e+01,  2.0035e+00,
         -1.3190e+01,  6.0009e+00, -9.3291e+00,  9.5986e+00,  6.7619e+00,
         -1.5659e+01,  7.9041e+00, -9.0815e+00, -1.2588e+00, -8.2280e+00,
         -3.9544e+00, -3.1830e+00, -4.9208e+00, -5.8930e+00,  2.3134e+00,
         -6.2999e+00, -2.9573e+00, -3.5331e+00, -3.5551e+00, -2.7225e+00,
         -5.7608e+00, -2.3866e+00, -1.6374e+00, -5.6490e+00, -2.0330e+00,
         -2.7544e+00, -6.2373e+00,  2.1661e-01, -5.5182e+00, -1.7449e+00,
         -3.6276e+00, -3.9632e+00, -3.4151e+00, -5.2902e+00, -4.7643e+00,
         -5.1784e+00, -4.1308e+00, -4.6408e+00, -3.5019e+00, -3.5599e+00,
         -3.6779e+00, -1.7927e+00, -2.3405e+00, -1.1088e+00, -2.0277e+00,
         -2.5763e+00, -2.6148e+00, -1.9386e+00,  1.3535e+00,  2.6257e+00,
          4.9568e+00,  5.3606e+00,  4.5571e+00,  4.2201e+00,  3.9949e+00,
          3.3130e+00,  3.5915e+00,  2.

In [89]:
model.eval()
with torch.no_grad():
    predictedd = model(file)

In [90]:
output = predictedd.numpy()[0].astype(int)

In [91]:
labelsss[output.argmax()]

'7masa'

In [92]:
torch.save(model.state_dict(), 'C:\\Users\\YAMAMA\\OneDrive\\سطح المكتب\\deploy\\models\\Sound_Classefier_model.pth')

In [3]:
loaded_model = torch.load('C:\\Users\\YAMAMA\\OneDrive\\سطح المكتب\\deploy\\models\\Sound_Classefier_model.pth')

AttributeError: Can't get attribute 'modell' on <module '__main__'>